<a href="https://colab.research.google.com/github/RomuloAlves2507/TrabalhoIA_unila_/blob/main/trabalhoIA_Unila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabalho de IA para Engenharia, tem como objetivo desenvolver e avaliar modelos de machine learning para diagnosticar a condição operacional de quatro componentes críticos (resfriador, válvula, bomba e acumulador) de um sistema hidráulico, utilizando dados brutos de múltiplos sensores.

Condition monitoring of hydraulic systems Data Set at ZeMA
     https://zenodo.org/records/1323611#.XfzEAEFCeUm

In [1]:
#import das bibliotecas necessárias
import pandas as pd
import pyarrow
from google.colab import files
import io

In [6]:
 #Criar arquivo parket a partir de 18 arquivos .txt (17 reais de sensores e 1 arquivo alvo)
def processar_arquivos_sensores(uploaded_files):
    print("Processando arquivos de SENSORES...")
    lista_de_dados_dos_arquivos = []
    numero_de_linhas_esperado = -1

    nomes_dos_arquivos = sorted(uploaded_files.keys())

    for filename in nomes_dos_arquivos:
        content = uploaded_files[filename].decode('utf-8')
        linhas = io.StringIO(content).readlines()

        if numero_de_linhas_esperado == -1:
            numero_de_linhas_esperado = len(linhas)
        elif len(linhas) != numero_de_linhas_esperado:
            print(f" ERRO: Arquivos de sensor com números de linhas diferentes!")
            print(f"Esperado: {numero_de_linhas_esperado}, Arquivo {filename} tem: {len(linhas)}")
            return None

    if numero_de_linhas_esperado == 0:
        print("Aviso: Os arquivos de sensores estão vazios.")
        return None

    print(f"Validação OK: Todos os {len(nomes_dos_arquivos)} arquivos de sensor têm {numero_de_linhas_esperado} linhas.")

    dados_combinados_finais = []

    for i in range(numero_de_linhas_esperado):
        linha_final_combinada = []
        for dados_arquivo_em_memoria in lista_de_dados_dos_arquivos:
            pass
        pass

    lista_de_dados_em_memoria = []
    for filename in nomes_dos_arquivos:
         content = uploaded_files[filename].decode('utf-8')
         linhas = io.StringIO(content).readlines()
         lista_de_dados_em_memoria.append(linhas)

    dados_combinados_finais = []
    for i in range(numero_de_linhas_esperado):
        linha_final_combinada = []
        for dados_arquivo in lista_de_dados_em_memoria:
            linha_atual = dados_arquivo[i].strip()
            valores_split = linha_atual.split()

            try:
                valores_float = []
                for val in valores_split:
                    val_corrigido = val.replace(',', '.')
                    valores_float.append(float(val_corrigido))
                linha_final_combinada.extend(valores_float)
            except ValueError as e:
                print(f" ERRO DE DADOS (Sensor) ")
                print(f"Linha: {i+1}, Valor: '{val}'")
                return None

        dados_combinados_finais.append(linha_final_combinada)

    print("Processamento de sensores concluído.")
    # Renomeia as colunas dos sensores para "f_0", "f_1", ... (feature_0, feature_1)
    df_sensores = pd.DataFrame(dados_combinados_finais)
    df_sensores.columns = [f'f_{i}' for i in range(len(df_sensores.columns))]

    return df_sensores, numero_de_linhas_esperado

def processar_arquivo_profile(uploaded_file, linhas_esperadas):
    print("Processando arquivo PROFILE...")

    if len(uploaded_file) != 1:
        print("ERRO: Por favor, selecione exatamente UM arquivo profile.txt")
        return None

    filename = list(uploaded_file.keys())[0]
    content = uploaded_file[filename].decode('utf-8')
    linhas = io.StringIO(content).readlines()

    if len(linhas) != linhas_esperadas:
        print(f"ERRO DE SINCRONIZAÇÃO")
        print(f"Os arquivos de sensor têm {linhas_esperadas} linhas, mas o profile.txt tem {len(linhas)} linhas.")
        return None

    print("Validação OK: O arquivo profile tem o mesmo número de linhas dos sensores.")

    dados_profile = []
    for i, linha_texto in enumerate(linhas):
        linha_atual = linha_texto.strip().split()

        # O profile.txt tem valores inteiros, não float
        try:
            # Converte para int
            valores_int = [int(float(v.replace(',', '.'))) for v in linha_atual]
            dados_profile.append(valores_int)
        except ValueError as e:
            print(f"ERRO DE DADOS (Profile)")
            print(f"Linha: {i+1}, Valor: '{linha_atual}'")
            return None

    # Nomes das colunas alvo
    colunas_alvo = ['resfriador', 'valvula', 'bomba', 'acumulador', 'stable_flag']

    df_profile = pd.DataFrame(dados_profile, columns=colunas_alvo[:len(dados_profile[0])])

    # Ajusta os nomes das colunas se o profile tiver menos de 5 colunas
    if len(df_profile.columns) < 5:
        df_profile.columns = colunas_alvo[:len(df_profile.columns)]

    print("Processamento do profile concluído.")
    return df_profile

def combinar_parquet():
    try:
        #Carregar Arquivos de Sensores (Features - X) ---
        print("ETAPA 1: SENSORES (FEATURES)")
        print("Por favor, selecione os 17 arquivos .txt dos SENSORES (PS1, TS1, CE, etc.)")
        uploaded_sensores = files.upload()
        if not uploaded_sensores:
            print("Nenhum arquivo de sensor selecionado. Operação cancelada.")
            return

        df_X, num_linhas = processar_arquivos_sensores(uploaded_sensores)
        if df_X is None:
            return

        #Carregar Arquivo de Alvo (Target - y)
        print("\n ETAPA 2: ALVO (TARGET)")
        print("Por favor, selecione o arquivo .txt de ALVO (profile.txt)")
        uploaded_profile = files.upload()
        if not uploaded_profile:
            print("Nenhum arquivo de alvo selecionado. Operação cancelada.")
            return

        df_y = processar_arquivo_profile(uploaded_profile, num_linhas)
        if df_y is None:
            return

        print("\n ETAPA 3: COMBINANDO DADOS")

        # Concatena os DataFrames lado a lado (axis=1)
        df_final_combinado = pd.concat([df_X, df_y], axis=1)

        print(f"Sucesso! Dados combinados.")
        print(f"Shape Final: {df_final_combinado.shape}")
        print("Primeiras 5 linhas e últimas 5 colunas (para conferir):")
        print(df_final_combinado.iloc[:5, -5:])

        #Salvar e Baixar o Parquet
        output_filename = 'dados_completos.parquet'
        print(f"\nSalvando arquivo Parquet como: {output_filename}")

        df_final_combinado.to_parquet(output_filename, engine='pyarrow', index=False)

        print(f"\n SUCESSO! :) ")
        print(f"Iniciando o download de '{output_filename}' para seu computador...")
        files.download(output_filename)

    except Exception as e:
        print(f"\n OCORREU UM ERRO ")
        print(f"Erro: {e}")


In [5]:
combinar_parquet()

--- ETAPA 1: SENSORES (FEATURES) ---
Por favor, selecione os 17 arquivos .txt dos SENSORES (PS1, TS1, CE, etc.)


Saving CE.txt to CE.txt
Saving CP.txt to CP.txt
Saving EPS1.txt to EPS1.txt
Saving FS1.txt to FS1.txt
Saving FS2.txt to FS2.txt
Saving PS1.txt to PS1.txt
Saving PS2.txt to PS2.txt
Saving PS3.txt to PS3.txt
Saving PS4.txt to PS4.txt
Saving PS5.txt to PS5.txt
Saving PS6.txt to PS6.txt
Saving SE.txt to SE.txt
Saving TS1.txt to TS1.txt
Saving TS2.txt to TS2.txt
Saving TS3.txt to TS3.txt
Saving TS4.txt to TS4.txt
Saving VS1.txt to VS1.txt
Processando arquivos de SENSORES...
Validação OK: Todos os 17 arquivos de sensor têm 2205 linhas.
Processamento de sensores concluído.

--- ETAPA 2: ALVO (TARGET) ---
Por favor, selecione o arquivo .txt de ALVO (profile.txt)


Saving profile.txt to profile.txt
Processando arquivo PROFILE...
Validação OK: O arquivo profile tem o mesmo número de linhas dos sensores.
Processamento do profile concluído.

--- ETAPA 3: COMBINANDO DADOS ---
Sucesso! Dados combinados.
Shape Final: (2205, 43685)
Primeiras 5 linhas e últimas 5 colunas (para conferir):
   resfriador  valvula  bomba  acumulador  stable_flag
0           3      100      0         130            1
1           3      100      0         130            1
2           3      100      0         130            1
3           3      100      0         130            1
4           3      100      0         130            1

Salvando arquivo Parquet como: dados_completos.parquet

--- SUCESSO! ---
Iniciando o download de 'dados_completos.parquet' para seu computador...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>